In [1]:
import pandas as pd
from lxml import html
import requests

In [2]:
from bs4 import BeautifulSoup
import urllib3
import requests
from lxml import html


def get_website():
    url = 'http://rosettacode.org/wiki/Copy_a_string'
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'lxml')
    return soup


def get_links():
    prefix = 'http://rosettacode.org'
    find_link = soup.select('td')
    return [prefix + line.a["href"] for line in find_link]

        
def get_title(soup):
    titles = []
    posts = soup.select("h2 > span")
    for post in posts:
        if post.text != "[edit]":
            titles.append(post.text)
    return titles

In [3]:
def get_code_examples(soup, languages):
    language_list = dict((x, []) for x in languages)
    lines = soup.select('pre')
    for line in lines:
        if line.has_attr("class"):
            for key, value in language_list.items():
                if line['class'][0] == key.lower():
                    language_list[key].append(line.text)
    return language_list

In [4]:
soupy = get_website()

In [5]:
langy = get_title(soupy)

In [6]:
dic = get_code_examples(soupy,langy)

In [7]:
df = pd.DataFrame(list(dic.items()))

In [9]:
df.head()

,0,1
0,Liberty BASIC,[]
1,Aime,[]
2,NewLISP,[(define (assert f msg) (if (not f) (println m...
3,Delphi,[program CopyString; {$APPTYPE CONSOLE} var s...
4,AutoHotkey,"[src := ""Hello""dst := src]"


In [11]:
class Classifier():
    def initialize():
        data = {}
        totals = Hash.new(1)
    
    
    def words(code):
        code.split(/[^a-z]/).reject{|w| w.empty?}
    
    
    def train(code,lang):
        totals[lang] += 1
        data[lang] ||= Hash.new(1)
        words(code).each {|w| @data[lang][w] += 1 }
        
    def classify(code):
        ws = words(code)
        data.keys.max_by do |lang|
      # We really want to multiply here but I use logs 
      # to avoid floating point underflow
      # (adding logs is equivalent to multiplication)
    Math.log(@totals[lang]) + ws.map{|w| Math.log(@data[lang][w])}.reduce(:+)


# Example usage

c = Classifier

# Train from files
c.train(open("code.rb").read, :ruby)
c.train(open("code.py").read, :python)
c.train(open("code.cs").read, :csharp)

# Test it on another file
c.classify(open("code2.py").read) # => :python (hopefully)

SyntaxError: invalid syntax (<ipython-input-11-f939bf4bbb85>, line 8)